In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip -q install transformers
!pip -q install datasets
!pip -q install simpletransformers
!pip -q install pythainlp
!pip install beautifulsoup4
!pip install tltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pythainlp.tag.tltk import get_ner
import requests
from bs4 import BeautifulSoup
from simpletransformers.question_answering import QuestionAnsweringModel
from pythainlp.tokenize import word_tokenize
import numpy as np 
bert = QuestionAnsweringModel("xlmroberta", " {Path/Model} ")

def Find_Keyword(data):
    keyword = []
    ner = get_ner(data)
    print(ner)
    for i in ner:
        if i[1] == 'NOUN' or i[1] == 'PROPN' :
            keyword.append(i[0])
    return keyword


def search(txt):
    url = f"https://th.wikipedia.org/wiki/{txt}"
 
    # Fetch URL Content
    r = requests.get(url)
    
    # Get body content
    soup = BeautifulSoup(r.text,'html.parser').select('body')[0]
    
    # Initialize variable
    paragraphs = []
    heading = []
    remaining_content = []
    
    # Iterate through all tags
    for tag in soup.find_all():
        
        # Check each tag name
        # For Paragraph use p tag
        if tag.name=="p":
        
            # use text for fetch the content inside p tag
            paragraphs.append(tag.text)
    paraGr = ''
    contextL = []
    for i in paragraphs:
        x = x+i.strip()
    # x = x.join(word_tokenize(x))
    ls = [77,1000,2000,3000,3500,4000,4500,5000,5500,6000]
    for i in range(len(ls)-1):
      contextL.append(paraGr[ls[i]:ls[i+1]])
    return contextL

def QA(data,qas):
  try:
    datals = []
    b_answer = []
    b_idx = []
    ls_check = []
    prob_check =[]
    txt_check =[]
    datas = data
    ques = qas
    for k in datas:
      for l in k:
          if l != '':
              
              question, text = ques, l
              print(question)
              print(text)
              input = [
                      {
                          "context": text,
                          "qas": [
                              {
                                  "question": question,
                                  "id": "0",
                              }
                          ],
                      }
                  ]
              txt, pb = bert.predict(input)
              datals.append(txt)
              datals.append(pb)
              print(txt)
              print(pb)
    for i in range(len(datals),2):
        if datals[i][0]['answer'][0] != '':
            ls_check.append(datals[i][0]['answer'][0])

    if len(ls_check) == 1:
            return ls_check[0]
    
    else:
        
    
      if len(ls_check) > 1:
          for j in range(1,len(datals),2):
              prob_check.append(datals[j][0]['probability'][0])
              txt_check.append(datals[j-1][0]['answer'][0])
          max_id = np.argmax(prob_check)
          ls_check.append(txt_check[max_id])
          return ls_check[-1]
          

      if len(ls_check) < 1:
          for j in range(1,len(datals),2):
            if len(datals[j][0]['probability']) > 1:
              prob_check.append(datals[j][0]['probability'][1])
              txt_check.append(datals[j-1][0]['answer'][1])

          if prob_check != []:
            max_id = np.argmax(prob_check)
            ls_check.append(txt_check[max_id])
            b_answer.append(ls_check[-1])
            b_idx.append(prob_check[max_id])

          else:
            return 'alice ไม่มีข้อมูลมากพอจะตอบคำถามนี้ได้ค่ะ'
      idx = np.argmax(b_idx)
      ans = b_answer[idx]
      return ans

  except:
    return 'ขอภัย Alice ไม่สามารถหาคำตอบได้'

In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import json
import numpy as np 

app = FastAPI()

class UserRequestIn(BaseModel):
    text: str

@app.post("/correctText")


def correct_text(user_request_in: UserRequestIn):
  try:
    txt_qas = user_request_in.text
    x = Find_Keyword(txt_qas)
    x.append(txt_qas)
    # x = x.split(',')
    # x = list(x)
    ques = x[-1]
    print(ques)
    datas = []

    for j in x[:-1]:
        txt = search(j)
        datas.append(txt)

    ans = QA(datas,ques)
    
    return ans
  except:
    return 'ขออภัย alice ไม่พบคำตอบ'
 
      


In [5]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [78]
INFO:uvicorn.error:Finished server process [78]
